In [1]:
!pip install simplejson
!pip install langchain-openai
!pip install chromadb
!pip install langchain
!pip install pathlib
!pip install pprintpp
!pip install jq
import json
import os

You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/charithesh/Workspace/Spring 2024/NLP/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
data_dir = "./data/"

In [9]:
DEFAULT_SYSTEM_PROMPT = "You are helpful medical assistance chatbot who assists the patients with their query."

def create_dataset(question, answer):
    return {
        "messages": [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
    }

def get_user(question):
    return {"role":"user","content":question}

def get_answer(answer):
    return {"role":"assistant","content":answer}

def get_system():
    return {"role":"system","content":DEFAULT_SYSTEM_PROMPT}
s=""
with open("train.jsonl","w") as file1:
    with open(os.path.join(data_dir,"combined.json"),"r") as f:
        temp = json.load(f)[0]
        for case in temp:
            if len(case["utterances"])%2==0:
                idx = 0
                message = {
                    "messages": [
                        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
                    ]
                }
                while idx < len(case["utterances"]):
                    message["messages"].append(get_user(case["utterances"][idx].split(":",1)[1]))
                    message["messages"].append(get_answer(case["utterances"][idx+1].split(":",1)[1]))
                    idx+=2
                ting = json.dumps(message)
                file1.write(ting+"\n")
print("completed")
            

completed


In [10]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPEN_AI_KEY"

In [12]:
import os
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
)

In [13]:
training_file_id = client.files.create(
  file=open("train.jsonl", "rb"),
  purpose="fine-tune"
)

print(f"Training File ID: {training_file_id}")

Training File ID: FileObject(id='file-kpkqUNx48JYG86NW93gVPtnQ', bytes=54698, created_at=1713122034, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [14]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id.id, 
  model="gpt-3.5-turbo-0125", 
  hyperparameters={
    "n_epochs": 1,
	"batch_size": 3,
	"learning_rate_multiplier": 0.3
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-v5TbBJDi1tNwFNX2bdCs7L3J.
Training Response: FineTuningJob(id='ftjob-v5TbBJDi1tNwFNX2bdCs7L3J', created_at=1713122102, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=3, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-WEQAhZs0V3APX6MtZLmaOjwC', result_files=[], status='validating_files', trained_tokens=None, training_file='file-kpkqUNx48JYG86NW93gVPtnQ', validation_file=None, user_provided_suffix=None, seed=2009165422, integrations=[])
Training Status: validating_files


In [22]:
import signal
import datetime


def signal_handler(sig, frame):
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return


print(f"Streaming events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

Streaming events for the fine-tuning job: ftjob-v5TbBJDi1tNwFNX2bdCs7L3J
2024-04-14 15:15:25 Fine-tuning job started
2024-04-14 15:15:24 Files validated, moving job to queued state
2024-04-14 15:15:02 Validating training file: file-kpkqUNx48JYG86NW93gVPtnQ
2024-04-14 15:15:02 Created fine-tuning job: ftjob-v5TbBJDi1tNwFNX2bdCs7L3J


In [24]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"Finetune job {job_id} finished with status: {status}")
print("Checking other finetune jobs in the subscription.")
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

Finetune job ftjob-v5TbBJDi1tNwFNX2bdCs7L3J finished with status: succeeded
Checking other finetune jobs in the subscription.
Found 1 finetune jobs.


In [26]:
# Retrieve the finetuned model
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)



ft:gpt-3.5-turbo-0125:personal::9Dzevxza


In [31]:
import openai

In [36]:
new_prompt = """ Ive had a cold which started on Christmas eve but appeared to be getting better over the following week.
However I now have what I think may be sinusitis - pain in the head, yellow mucus from the nose and stuffiness -squeaking from the sinuses. 
Will this go away on its own or should I see my GP?"""

completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages= [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": new_prompt},
        ]
)

print(completion.choices[0].message.content)

 Hello, I'm sorry to hear that you're not feeling well. Sinusitis can be a complication of a cold, as you've described. Sinusitis can be caused by inflammation and congestion of the sinus cavities, usually due to an infection. While some mild cases of sinusitis may improve on their own, more severe cases may require medical treatment. 

Considering the color of mucus and your symptoms, it is advisable to consult a doctor.
They may prescribe antibiotics or other treatments to help clear up the infection and relieve your symptoms. In the meantime, you can use over-the-counter decongestants and pain relievers to help manage your symptoms. 

If you are experiencing severe symptoms such as high fever, severe headache, or difficulty breathing, it's essential to seek medical attention promptly.
